In [ ]:
import pandas as pd

file_path = '/content/Inputs-Targets.csv'
weather_data = pd.read_csv(file_path)

print(weather_data.head())

missing_values = weather_data.isnull().sum()
print("Missing Values:\n", missing_values)

statistical_summary = weather_data.describe()
print("Statistical Summary:\n", statistical_summary)

target_distribution = weather_data[['7P', '14P', '28P']].apply(pd.Series.value_counts)
print("Target Distribution:\n", target_distribution)

correlation_matrix = weather_data.corr()
print("Correlation Matrix:\n", correlation_matrix.iloc[:10, :10])


          Date     AlfaX     AlfaY     AlfaZ     AlfaD    BravoX    BravoY  \
0  1874-Mar-19 -0.387448 -0.004925  0.035286  0.389083  0.702318  0.178241   
1  1874-Mar-20 -0.391974 -0.031792  0.033512  0.394686  0.696998  0.197682   
2  1874-Mar-21 -0.394613 -0.058506  0.031577  0.400175  0.691137  0.216970   
3  1874-Mar-22 -0.395431 -0.084952  0.029497  0.405527  0.684740  0.236090   
4  1874-Mar-23 -0.394494 -0.111021  0.027285  0.410726  0.677812  0.255027   

     BravoZ    BravoD  CharlieX  ...      PapaY     PapaZ      PapaD  \
0 -0.038235  0.725591 -0.993904  ...  13.967170  0.209719  17.842542   
1 -0.037669  0.725467 -0.994185  ...  13.981678  0.209753  17.855734   
2 -0.037073  0.725342 -0.994298  ...  13.996177  0.209787  17.869108   
3 -0.036448  0.725215 -0.994218  ...  14.010663  0.209821  17.882659   
4 -0.035795  0.725086 -0.993915  ...  14.025129  0.209855  17.896382   

     QuebecX    QuebecY   QuebecZ    QuebecD  7P  14P  28P  
0  26.853693  14.828570 -0.900904  30

<ipython-input-2-94b02ff9981d>:17: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  correlation_matrix = weather_data.corr()


Correlation Matrix:
              AlfaX     AlfaY     AlfaZ     AlfaD    BravoX    BravoY  \
AlfaX     1.000000  0.012079 -0.740589 -0.224509 -0.000480  0.000002   
AlfaY     0.012079  1.000000  0.662963 -0.977112 -0.000368 -0.000791   
AlfaZ    -0.740589  0.662963  1.000000 -0.488534  0.000113 -0.000534   
AlfaD    -0.224509 -0.977112 -0.488534  1.000000  0.000460  0.000770   
BravoX   -0.000480 -0.000368  0.000113  0.000460  1.000000  0.001362   
BravoY    0.000002 -0.000791 -0.000534  0.000770  0.001362  1.000000   
BravoZ    0.000467  0.000177 -0.000232 -0.000271 -0.973555  0.227117   
BravoD   -0.000322  0.000343  0.000473 -0.000265  0.662736 -0.747929   
CharlieX  0.000139  0.000769  0.000412 -0.000779 -0.000132 -0.003357   
CharlieY -0.000267 -0.000421 -0.000084  0.000467  0.003198  0.000193   

            BravoZ    BravoD  CharlieX  CharlieY  
AlfaX     0.000467 -0.000322  0.000139 -0.000267  
AlfaY     0.000177  0.000343  0.000769 -0.000421  
AlfaZ    -0.000232  0.000473  0.0

# Part 1: Explore Data - EDA

1. Missing Values

  No missing values were found, which makes the task a little easier to deal with <br>

2. Distribution of target variables

  The target variables are balanced in general but their distributions are different <br>
3. Observing the statistical summary of the data
  - The predictive features vary in scale and distribution, which calls for the need for scaling.
  - The variety in scales also suggests that the features are related to geographic or atmospheric measurements.
4. Looking for correlations between the features

  The subset of the correlation matrix shows that the features have strong correlations with each other. The full matrix needs more analysis to understand all interactions.
  



# Part 3: Deciding how to deal with multiple targets:

There are multiple ways we can deal with multiple targets.
1. Train Separate models for each target: this might give a higher accuracy since each model would be specialized for each target, however, it is time-consuming and resource-intensive.
2. Tune one model using one target and apply it to the other targets: This approach is more reasonable since the targets are fairly similar. It would save time and resources.

However, since the Extra credit requirements are asking to test each target separately, I will use multiple models to try predicting the rainfall.

# Part 2: Deciding which models to use

7-day Rainfall: LSTM
- Since 7 days is a short period, I am using the LSTM model (Long Short-Term Memory). These models are suitable for short dependencies in time-series data.

14-day Rainfall: GRU
- GRU is a simple and faster model to use on time-series data. It has fewer parameters and it is beneficial in predicting mid-range time series problems.

28-day Rainfall: Random Forest (Machine Learning model)
- Since it is a longer period, I will be using a random forest. The importance of specific sequential information is not as important in longer periods. A random forest is a good choice to capture complex non-linear relationships across multiple features.




#First Model: LSTM

To test this model, some steps need to be considered before training the model. <br>
1. Data preprocessing: Scaling or normalizing, since neural networks are sensitive to the scale of the input data
2. Preparing sequence: creating lag features and reshaping the data into a format suitable for LSTMs.
3. Tuning the model: experimenting with different numbers of LSTM units, layers, and dropout rates.

In [ ]:
import pandas as pd
from keras.models import Sequential
from keras.layers import LSTM, Dense
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split

data = pd.read_csv('/content/Inputs-Targets.csv')

data['Date'] = pd.to_datetime(data['Date'])
data['Year'] = data['Date'].dt.year
data['Month'] = data['Date'].dt.month

X = data.drop(['Date', '7P', '14P', '28P'], axis=1)
y = data['7P']

encoder = OneHotEncoder(sparse_output=False)
y_encoded = encoder.fit_transform(y.values.reshape(-1, 1))

X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

X_train_reshaped = X_train_scaled.reshape((X_train_scaled.shape[0], 1, X_train_scaled.shape[1]))
X_test_reshaped = X_test_scaled.reshape((X_test_scaled.shape[0], 1, X_test_scaled.shape[1]))

model = Sequential()
model.add(LSTM(50, input_shape=(X_train_reshaped.shape[1], X_train_reshaped.shape[2])))
model.add(Dense(5, activation='softmax'))  # Output layer for 5 categories

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(X_train_reshaped, y_train, epochs=10, batch_size=32, validation_data=(X_test_reshaped, y_test), verbose=2)


Epoch 1/10
1364/1364 - 6s - loss: 1.5217 - accuracy: 0.3369 - val_loss: 1.5027 - val_accuracy: 0.3550 - 6s/epoch - 5ms/step
Epoch 2/10
1364/1364 - 5s - loss: 1.4794 - accuracy: 0.3658 - val_loss: 1.4753 - val_accuracy: 0.3675 - 5s/epoch - 4ms/step
Epoch 3/10
1364/1364 - 4s - loss: 1.4436 - accuracy: 0.3904 - val_loss: 1.4479 - val_accuracy: 0.3907 - 4s/epoch - 3ms/step
Epoch 4/10
1364/1364 - 4s - loss: 1.4094 - accuracy: 0.4133 - val_loss: 1.4257 - val_accuracy: 0.4012 - 4s/epoch - 3ms/step
Epoch 5/10
1364/1364 - 5s - loss: 1.3787 - accuracy: 0.4283 - val_loss: 1.4111 - val_accuracy: 0.4106 - 5s/epoch - 4ms/step
Epoch 6/10
1364/1364 - 4s - loss: 1.3524 - accuracy: 0.4463 - val_loss: 1.3885 - val_accuracy: 0.4289 - 4s/epoch - 3ms/step
Epoch 7/10
1364/1364 - 4s - loss: 1.3284 - accuracy: 0.4596 - val_loss: 1.3766 - val_accuracy: 0.4296 - 4s/epoch - 3ms/step
Epoch 8/10
1364/1364 - 4s - loss: 1.3068 - accuracy: 0.4695 - val_loss: 1.3634 - val_accuracy: 0.4384 - 4s/epoch - 3ms/step
Epoch 9/

In [ ]:
print("X_train_reshaped shape:", X_train_reshaped.shape)
print("X_test_reshaped shape:", X_test_reshaped.shape)


X_train_reshaped shape: (43635, 1, 70)
X_test_reshaped shape: (10909, 1, 70)


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt

y_pred = model.predict(X_test_reshaped)
y_pred_classes = y_pred.argmax(axis=1)
y_true = y_test.argmax(axis=1)

print(confusion_matrix(y_true, y_pred_classes))

print(classification_report(y_true, y_pred_classes))

# # Plotting learning curves
# plt.figure(figsize=(12, 4))
# plt.subplot(1, 2, 1)
# plt.plot(model.history.history['accuracy'], label='Training Accuracy')
# plt.plot(model.history.history['val_accuracy'], label='Validation Accuracy')
# plt.title('Accuracy over Epochs')
# plt.legend()

# plt.subplot(1, 2, 2)
# plt.plot(model.history.history['loss'], label='Training Loss')
# plt.plot(model.history.history['val_loss'], label='Validation Loss')
# plt.title('Loss over Epochs')
# plt.legend()
# plt.show()


341/341 [==============================] - 4s 5ms/step
[[1991  358  146  131  365]
 [ 715  677  205  163  353]
 [ 385  255  339  222  334]
 [ 408  244  205  411  536]
 [ 384  172  155  229 1526]]
              precision    recall  f1-score   support

           0       0.51      0.67      0.58      2991
           1       0.40      0.32      0.35      2113
           2       0.32      0.22      0.26      1535
           3       0.36      0.23      0.28      1804
           4       0.49      0.62      0.55      2466

    accuracy                           0.45     10909
   macro avg       0.42      0.41      0.40     10909
weighted avg       0.43      0.45      0.43     10909



# Initial Results:
1. Accuracy: There are improvements in both training and validation accuracy over the epochs, which means that the model is learning correctly.
2. Validation Accuracy: It is increasing which means that the model is succeeding in generalizing to unseen data. However, the maximum value for validation accuracy is 45%, which means that the model could be improved.
3. Loss Reduction: the losses are decreasing, which is another indicator that the model is improving in performance. More training could take place since the values for the losses are still not low enough.

# Interpretation of confusion matrix and Classification Report Analysis:
- The confusion matrix suggests that the data might be imbalanced since the numbers across the rows and columns are not evenly distributed.
- The classification report shows that categories 0 and 4 have the highest recall, which means that the model is better at identifying these categories. F1 score is also the highest for 0 and 4. The overall accuracy of 0.45 means that the model correctly predicts the category 45% of the time across all categories.

#Next Steps:
1. Increasing the number of epochs
2. Tuning the hyperparameters of the model including LSTM units, learning rate, batch size, and dropout layers.
3. Cross-validation

The following code applied the following modifications: <br>
1. Increases LSM units: it is increased to 100, which allows the model to capture more complex patterns
2. Additional LSTM layer and dropout: This is helpful for regularization to prevent overfitting.
3. Adjusted Learning rate: I set it to 0.001. This is a commonly used default value.
4. Increased epochs and batch size: I increased both of them, which allows the model more opportunity to learn from the data and can help in achieving better convergence.
5. Return sequence: I set this parameter to "True" to connect the first layer to the subsequent LSTM layer.


In [ ]:
import pandas as pd
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import Adam

data = pd.read_csv('/content/Inputs-Targets.csv')

data['Date'] = pd.to_datetime(data['Date'])
data['Year'] = data['Date'].dt.year
data['Month'] = data['Date'].dt.month

X = data.drop(['Date', '7P', '14P', '28P'], axis=1)
y = data['7P']

encoder = OneHotEncoder(sparse=False)
y_encoded = encoder.fit_transform(y.values.reshape(-1, 1))

X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

X_train_reshaped = X_train_scaled.reshape((X_train_scaled.shape[0], 1, X_train_scaled.shape[1]))
X_test_reshaped = X_test_scaled.reshape((X_test_scaled.shape[0], 1, X_test_scaled.shape[1]))

model = Sequential()
model.add(LSTM(100, return_sequences=True, input_shape=(X_train_reshaped.shape[1], X_train_reshaped.shape[2])))  # Increased number of LSTM units
model.add(Dropout(0.2))  # Dropout for regularization
model.add(LSTM(50, return_sequences=False))  # Additional LSTM layer
model.add(Dropout(0.2))  # Dropout for regularization
model.add(Dense(5, activation='softmax'))  # Output layer for 5 categories

optimizer = Adam(learning_rate=0.001)  # Adjusted learning rate
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

model.fit(X_train_reshaped, y_train, epochs=100, batch_size=64, validation_data=(X_test_reshaped, y_test), verbose=2)  # Increased epochs and adjusted batch size


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Epoch 1/100
682/682 - 10s - loss: 1.5284 - accuracy: 0.3376 - val_loss: 1.5120 - val_accuracy: 0.3520 - 10s/epoch - 14ms/step
Epoch 2/100
682/682 - 4s - loss: 1.4952 - accuracy: 0.3569 - val_loss: 1.4872 - val_accuracy: 0.3635 - 4s/epoch - 6ms/step
Epoch 3/100
682/682 - 5s - loss: 1.4686 - accuracy: 0.3742 - val_loss: 1.4617 - val_accuracy: 0.3791 - 5s/epoch - 7ms/step
Epoch 4/100
682/682 - 4s - loss: 1.4403 - accuracy: 0.3917 - val_loss: 1.4352 - val_accuracy: 0.3974 - 4s/epoch - 6ms/step
Epoch 5/100
682/682 - 4s - loss: 1.4117 - accuracy: 0.4082 - val_loss: 1.4086 - val_accuracy: 0.4119 - 4s/epoch - 6ms/step
Epoch 6/100
682/682 - 5s - loss: 1.3863 - accuracy: 0.4222 - val_loss: 1.3849 - val_accuracy: 0.4271 - 5s/epoch - 8ms/step
Epoch 7/100
682/682 - 4s - loss: 1.3622 - accuracy: 0.4347 - val_loss: 1.3613 - val_accuracy: 0.4412 - 4s/epoch - 6ms/step
Epoch 8/100
682/682 - 4s - loss: 1.3410 - accuracy: 0.4457 - val_loss: 1.3407 - val_accuracy: 0.4564 - 4s/epoch - 6ms/step
Epoch 9/100
6

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt

y_pred = model.predict(X_test_reshaped)
y_pred_classes = y_pred.argmax(axis=1)
y_true = y_test.argmax(axis=1)

# Confusion Matrix
print(confusion_matrix(y_true, y_pred_classes))

# Classification Report
print(classification_report(y_true, y_pred_classes))

341/341 [==============================] - 3s 5ms/step
[[2354  329  109   93  106]
 [ 388 1211  205  161  148]
 [ 201  187  754  213  180]
 [ 130   97  154 1018  405]
 [  46   41   47  215 2117]]
              precision    recall  f1-score   support

           0       0.75      0.79      0.77      2991
           1       0.65      0.57      0.61      2113
           2       0.59      0.49      0.54      1535
           3       0.60      0.56      0.58      1804
           4       0.72      0.86      0.78      2466

    accuracy                           0.68     10909
   macro avg       0.66      0.65      0.66     10909
weighted avg       0.68      0.68      0.68     10909



#Analysis
**Training and Validation Loss and Accuracy:**
- There is a steady decrease in loss and increase in accuracy over the 100 epochs, which is a good sign
- The validation accuracy has improved to 68.3%, which is a good increase from the previous model.
- There are no signs of overfitting since the validation loss and accuracy are in line with the training loss and accuracy

**Confusion Matrix:**
- The diagonal elements represent the true positive predictions for each class (0-4). They are higher than the previous model, which is a good sign.
- The misclassification showed a decrease as well, which means that the model is making fewer mistakes

**Classification Report Analysis:**

- Precision: precision is high for classes 0 and 4, which means there are fewer false positives. As for classes 1,2, and 3, they have moderate precision.
- Recall: Class 4 has the highest recall, which is 0.86. This means that the model is very effective in predicting this category. As for classes 0,1,2, and 3, they have moderate precision.
- F1 Score: This score combines precision and recall. The scorer is high for classes 0 and 4, which means that there is a good balance between precision and recall for these categories. The rest of the classes have moderate F1 scores.



**Next Steps:**

There are multiple steps that could be taken at this point to improve the performance of the model. These include more modifications to the model like increasing the number of epochs, adding additional LSTM layers, and dropout for regularization. <br>

However, for the next step, I am going to experiment with training the model using cross-validation, to ensure that the model is robust.

In [ ]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import TimeSeriesSplit
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report, confusion_matrix

data = pd.read_csv('/content/Inputs-Targets.csv')
data['Date'] = pd.to_datetime(data['Date'])
data['Year'] = data['Date'].dt.year
data['Month'] = data['Date'].dt.month
X = data.drop(['Date', '7P', '14P', '28P'], axis=1)
y = data['7P']
encoder = OneHotEncoder(sparse=False)
y_encoded = encoder.fit_transform(y.values.reshape(-1, 1))
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

def create_model(input_shape):
    model = Sequential()
    model.add(LSTM(100, return_sequences=True, input_shape=input_shape))
    model.add(Dropout(0.2))
    model.add(LSTM(50, return_sequences=False))
    model.add(Dropout(0.2))
    model.add(Dense(5, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=0.001), metrics=['accuracy'])
    return model

tscv = TimeSeriesSplit(n_splits=5)
fold_no = 1

for train_index, test_index in tscv.split(X_scaled):
    X_train, X_test = X_scaled[train_index], X_scaled[test_index]
    y_train, y_test = y_encoded[train_index], y_encoded[test_index]

    X_train_reshaped = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
    X_test_reshaped = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))

    model = create_model((X_train_reshaped.shape[1], X_train_reshaped.shape[2]))
    print(f'Training for fold {fold_no} ...')
    model.fit(X_train_reshaped, y_train, epochs=50, batch_size=32, verbose=2)

    scores = model.evaluate(X_test_reshaped, y_test, verbose=0)
    print(f'Score for fold {fold_no}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%')

    fold_no += 1


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Training for fold 1 ...
Epoch 1/50
285/285 - 6s - loss: 1.5035 - accuracy: 0.3675 - 6s/epoch - 20ms/step
Epoch 2/50
285/285 - 2s - loss: 1.4454 - accuracy: 0.4061 - 2s/epoch - 5ms/step
Epoch 3/50
285/285 - 2s - loss: 1.3961 - accuracy: 0.4296 - 2s/epoch - 6ms/step
Epoch 4/50
285/285 - 2s - loss: 1.3465 - accuracy: 0.4590 - 2s/epoch - 6ms/step
Epoch 5/50
285/285 - 1s - loss: 1.2953 - accuracy: 0.4823 - 1s/epoch - 5ms/step
Epoch 6/50
285/285 - 1s - loss: 1.2413 - accuracy: 0.5064 - 1s/epoch - 5ms/step
Epoch 7/50
285/285 - 1s - loss: 1.1899 - accuracy: 0.5255 - 1s/epoch - 5ms/step
Epoch 8/50
285/285 - 1s - loss: 1.1502 - accuracy: 0.5448 - 1s/epoch - 5ms/step
Epoch 9/50
285/285 - 1s - loss: 1.1074 - accuracy: 0.5634 - 1s/epoch - 5ms/step
Epoch 10/50
285/285 - 1s - loss: 1.0690 - accuracy: 0.5796 - 1s/epoch - 5ms/step
Epoch 11/50
285/285 - 1s - loss: 1.0340 - accuracy: 0.6014 - 1s/epoch - 5ms/step
Epoch 12/50
285/285 - 2s - loss: 1.0006 - accuracy: 0.6050 - 2s/epoch - 7ms/step
Epoch 13/50


# Results of using time-series cross-validation:

**Training performance:**
The training performance shows a decrease in loss and an increase in accuracy over the epochs. This means that the model is learning well. The model shows variability across folds, which is normal for time-series data.

**Cross Validation Scores:**
- The accuracy scores during cross-validation are relatively low (around 22-26%) This suggests that the model is not adequately capturing the complexity or patterns for accurate predictions.
- The model shows consistency of scores across the folds which suggests the the model's performance is not overly dependent on a specific subset of data. This leads to the conclusion that the challenge lies in the complexity of the task rather than using the wrong subset.

# Next Steps:

There are more steps that could be taken to improve the LSTM model. These include:

1. Combining LSTM and CNN layers to accommodate for the complexity of the data
2. Feature Engineering: Review and potentially enhance feature engineering by creating more meaningful features or transforming existing ones.
3. Address Class Imbalance: By using techniques like class wighting, oversampling, or undersampling.


However, for the time being, I will stop exploring the LSTM model and I will test other models on the other targets.

# Second Model: Random forest for 28P

In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import StandardScaler, OneHotEncoder

file_path = '/content/Inputs-Targets.csv'
data = pd.read_csv(file_path)


data['Date'] = pd.to_datetime(data['Date'])
data['Year'] = data['Date'].dt.year
data['Month'] = data['Date'].dt.month
X = data.drop(['Date', '7P', '14P', '28P'], axis=1)
y = data['28P']

encoder = OneHotEncoder(sparse_output=False)
y_encoded = encoder.fit_transform(y.values.reshape(-1, 1))

X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train_scaled, y_train)

y_pred = rf_model.predict(X_test_scaled)

print("Confusion Matrix:\n", confusion_matrix(y_test.argmax(axis=1), y_pred.argmax(axis=1)))
print("\nClassification Report:\n", classification_report(y_test.argmax(axis=1), y_pred.argmax(axis=1)))


Confusion Matrix:
 [[ 196   17    1    0    0]
 [  71 2406   95    6    2]
 [ 124  111 2110  122   16]
 [ 108   11  114 1786  138]
 [  61    2   10  123 3279]]

Classification Report:
               precision    recall  f1-score   support

           0       0.35      0.92      0.51       214
           1       0.94      0.93      0.94      2580
           2       0.91      0.85      0.88      2483
           3       0.88      0.83      0.85      2157
           4       0.95      0.94      0.95      3475

    accuracy                           0.90     10909
   macro avg       0.81      0.89      0.82     10909
weighted avg       0.91      0.90      0.90     10909



# Analysis:

**Confusion Matrix:**
- Class 0: It shows a high recall of 92% but a low precision of 35%. This model is good at identifying true positives but it also incorrectly classifies instances from other classes as class 0
- Class 1 to 4: They show higher precision and recall, which indicates better performance. Classes 1,2,3, and 4 show precision that is above 88%. The recall ranges between 83% and 94%, which means that the model is accurate for these classes.

**Classification Report Analysis:**
- As mentioned above, class 0 is the only class that shows a low F1 score. F1 scores help determine if there is a good balance between precision and recall. However, classes 1 to 4 show high F1 scores, which suggests a good balance.
- The overall accuracy of the model is 90%, which means that the model is accurately predicting the rainfall 90% of the time. <br>
- Macro Avg is 82% which is a good indicator of the model's performance across classes, especially if there is a class imbalance.<br>
Weighted Avg: The weighted average for the F1 score is 90%, which is pretty high considering the classes imbalance.


In [ ]:
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split

data = pd.read_csv('/content/Inputs-Targets.csv')
data['Date'] = pd.to_datetime(data['Date'])
data['Year'] = data['Date'].dt.year
data['Month'] = data['Date'].dt.month
X = data.drop(['Date', '7P', '14P', '28P'], axis=1)
y = data['14P']

encoder = OneHotEncoder(sparse_output=False)
y_encoded = encoder.fit_transform(y.values.reshape(-1, 1))

X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

X_train_reshaped = X_train_scaled.reshape((X_train_scaled.shape[0], 1, X_train_scaled.shape[1]))
X_test_reshaped = X_test_scaled.reshape((X_test_scaled.shape[0], 1, X_test_scaled.shape[1]))

model = Sequential()
model.add(GRU(50, input_shape=(X_train_reshaped.shape[1], X_train_reshaped.shape[2])))
model.add(Dense(y_encoded.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

history = model.fit(X_train_reshaped, y_train, epochs=50, batch_size=32, validation_data=(X_test_reshaped, y_test), verbose=2)

print(history.history)


Epoch 1/50
1364/1364 - 7s - loss: 1.4677 - accuracy: 0.3562 - val_loss: 1.4334 - val_accuracy: 0.3730 - 7s/epoch - 5ms/step
Epoch 2/50
1364/1364 - 4s - loss: 1.3979 - accuracy: 0.4031 - val_loss: 1.3939 - val_accuracy: 0.3960 - 4s/epoch - 3ms/step
Epoch 3/50
1364/1364 - 5s - loss: 1.3495 - accuracy: 0.4281 - val_loss: 1.3448 - val_accuracy: 0.4285 - 5s/epoch - 3ms/step
Epoch 4/50
1364/1364 - 4s - loss: 1.3054 - accuracy: 0.4549 - val_loss: 1.3152 - val_accuracy: 0.4444 - 4s/epoch - 3ms/step
Epoch 5/50
1364/1364 - 4s - loss: 1.2661 - accuracy: 0.4774 - val_loss: 1.2837 - val_accuracy: 0.4620 - 4s/epoch - 3ms/step
Epoch 6/50
1364/1364 - 4s - loss: 1.2325 - accuracy: 0.4935 - val_loss: 1.2573 - val_accuracy: 0.4790 - 4s/epoch - 3ms/step
Epoch 7/50
1364/1364 - 5s - loss: 1.2017 - accuracy: 0.5084 - val_loss: 1.2375 - val_accuracy: 0.4911 - 5s/epoch - 4ms/step
Epoch 8/50
1364/1364 - 5s - loss: 1.1764 - accuracy: 0.5240 - val_loss: 1.2103 - val_accuracy: 0.5010 - 5s/epoch - 3ms/step
Epoch 9/

# Analysis:
Training and Validation Loss:
- The validation and training loss decrease over 50 epochs, which is a good sign.
- The loss started at 1.469 and ended at 0.7947, while the validation loss started at 1.4377 and ended at 0.9061. The consistent decrease suggests that the model is generalizing well without an obvious overfitting.

Training and Validation Accuracy:
- The training accuracy started at 35.18% and increased to 96.61%, while validation accuracy started at 36.78% and stopped at 64.96%. The higher training accuracy compared to validation accuracy is normal. In general, the model shows a good level of generalization taking into account the nature of the task.

Potential Overfitting: The last epochs show a slightly higher gap between the training accuracy and the validation accuracy. This could be classified as an early sign of overfitting.

**Next Steps:**

1. Experiment with different hyperparamters
2. Implement Early Stopping


In [ ]:
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense, Dropout
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping

data = pd.read_csv('/content/Inputs-Targets.csv')

data['Date'] = pd.to_datetime(data['Date'])
data['Year'] = data['Date'].dt.year
data['Month'] = data['Date'].dt.month
X = data.drop(['Date', '7P', '14P', '28P'], axis=1)
y = data['14P']

encoder = OneHotEncoder(sparse=False)
y_encoded = encoder.fit_transform(y.values.reshape(-1, 1))

X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

X_train_reshaped = X_train_scaled.reshape((X_train_scaled.shape[0], 1, X_train_scaled.shape[1]))
X_test_reshaped = X_test_scaled.reshape((X_test_scaled.shape[0], 1, X_test_scaled.shape[1]))

model = Sequential()
model.add(GRU(70, input_shape=(X_train_reshaped.shape[1], X_train_reshaped.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(GRU(50))
model.add(Dropout(0.2))
model.add(Dense(y_encoded.shape[1], activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

history = model.fit(X_train_reshaped, y_train, validation_data=(X_test_reshaped, y_test),
                    epochs=50, batch_size=64, callbacks=[early_stopping], verbose=2)


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Epoch 1/50
682/682 - 10s - loss: 1.4870 - accuracy: 0.3392 - val_loss: 1.4517 - val_accuracy: 0.3571 - 10s/epoch - 15ms/step
Epoch 2/50
682/682 - 4s - loss: 1.4376 - accuracy: 0.3719 - val_loss: 1.4177 - val_accuracy: 0.3757 - 4s/epoch - 5ms/step
Epoch 3/50
682/682 - 5s - loss: 1.4077 - accuracy: 0.3905 - val_loss: 1.3819 - val_accuracy: 0.4065 - 5s/epoch - 7ms/step
Epoch 4/50
682/682 - 4s - loss: 1.3764 - accuracy: 0.4056 - val_loss: 1.3510 - val_accuracy: 0.4252 - 4s/epoch - 6ms/step
Epoch 5/50
682/682 - 4s - loss: 1.3458 - accuracy: 0.4249 - val_loss: 1.3129 - val_accuracy: 0.4385 - 4s/epoch - 6ms/step
Epoch 6/50
682/682 - 6s - loss: 1.3199 - accuracy: 0.4388 - val_loss: 1.2810 - val_accuracy: 0.4612 - 6s/epoch - 8ms/step
Epoch 7/50
682/682 - 4s - loss: 1.2956 - accuracy: 0.4513 - val_loss: 1.2498 - val_accuracy: 0.4758 - 4s/epoch - 5ms/step
Epoch 8/50
682/682 - 4s - loss: 1.2692 - accuracy: 0.4643 - val_loss: 1.2248 - val_accuracy: 0.4883 - 4s/epoch - 5ms/step
Epoch 9/50
682/682 - 

**In-depth Analysis of Rainfall Prediction Models:**

This report aims to analyze the performance of three distinct models- LSTM, Random Forest, and GRU - each model is applied to predict rainfall over time horizons (7, 14, 28) days. <br>

1. LSTM on 7-day Rainfall Prediction (7P):
  - The model showed gradual improvement over epochs which shows a learning trend
  - The accuracy plateaued, which means that the model might be underfitting or needs more complex network architecture

  I explored the LSTM model the most, and I tested the most modifications on it. In general, it performed well.


2. Random Forest on 28-Day Rainfall Prediction (28P):
  - This method of using ensembled provides valuable information without needing to use transformations required by neural networks.
  - High precision and recall scores were achieved, specifically in predicting majority classes, which is an indicator of its efficiency in handling imbalanced data.
  - Despite it being a strong, simple model, it is limited in capturing temporal dependencies, which are of importance for time-series forecasting.

3. GRU on 14-Day Rainfall Prediction (14P):
  - It is similar to the LSTM model but more efficient. I used early stopping and optimization of the hyperparameters to enhance performance.
  - It demonstrated the highest validation accuracy among all models, which suggests effectiveness for this task
  - Early stopping was successful in preventing overfitting


**Comparative Overview:** <br>
**Accuracy and Efficiency:** GRU achieved the highest accuracy. Random Forest, while less accurate, was easier and more efficient to implement. <br>
**Model Complexity:** LSTM and GRU are naturally more complex and they require careful tuning. Random Forest is a more straightforward approach. <br>
**Computational Resources:** LSTM and GRU require greater computational resources and time. <br>
**Overfitting:** Early stopping was effective in preventing overfitting using the GRU model. Random Forest avoids overfitting naturally. However, LSTM showed potential overfitting. <br>

**Conclusion:**<br>
Depending on the task that is needed from predicting the rainfall, GRU or Random Forest could be used as efficient models. For future advancements, hybrid models could be used to improve the performance.
